In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import re 
from glob import glob
import pandas as pd
import gradio as gr
import torch
import datasets
import json_repair
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# 모델 로드 경로 설정
peft_model_id = "/home/admin/.cache/.work/model/model_5e-05_alpha-128_r-256"

# 모델과 토크나이저 로드
model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_id,
    device_map="auto",
    torch_dtype=torch.float16
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

# 데이터셋 로드 및 준비
file_list = glob("./data/*.csv")
df = pd.concat([pd.read_csv(file) for file in file_list])


def get_chat_format(element):
    system_prompt = "너는 개인정보를 비식별화하는 Assistant야. 너는 주어진 데이터를 바탕으로 개인정보를 비식별화하는 작업을 해야해."

    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": element["origin_data"]},
            {"role": "assistant", "content": element["anonymized_data"]},
        ], 
        "label": element["mapping"]
    }

dataset = datasets.Dataset.from_pandas(df)
dataset = dataset.map(get_chat_format, remove_columns=dataset.features, batched=False)
dataset = dataset.shuffle(seed=42)
dataset = dataset.train_test_split(test_size=0.1, seed=42)


# 데이터셋에서 데이터 선택 함수
def select_data(index):
    data = dataset["test"][index]
    data['label'] = json_repair.loads(data['label'])
    label_str = "\n".join([f"{orig_val} -> {ph}" for orig_val, ph in data['label'].items()])
    return data["messages"][1]["content"], label_str



def extract_placeholder_mapping(original_text, transformed_text, allowed_types):
    allowed_pattern = re.compile(r'\[(' + '|'.join(allowed_types) + r')\d*\]')
    generic_pattern = re.compile(r'(\[[^]]+\])')

    mapping = {}

    orig_lines = original_text.splitlines()
    trans_lines = transformed_text.splitlines()
    n_lines = min(len(orig_lines), len(trans_lines))

    for idx in range(n_lines):
        orig_line = orig_lines[idx]
        trans_line = trans_lines[idx]

        parts = re.split(generic_pattern, trans_line)
        orig_pos = 0

        for i, part in enumerate(parts):
            if allowed_pattern.match(part):
                # placeholder 발견
                # 다음 literal을 찾음
                next_literal = parts[i + 1] if i + 1 < len(parts) else ''
                
                # 다음 literal이 존재하면, 그 literal까지의 텍스트를 추출
                if next_literal:
                    next_idx = orig_line.find(next_literal, orig_pos)
                    if next_idx != -1:
                        replaced_text = orig_line[orig_pos:next_idx]
                        orig_pos = next_idx
                    else:
                        # 다음 literal을 못 찾으면 끝까지
                        replaced_text = orig_line[orig_pos:]
                        orig_pos = len(orig_line)
                else:
                    # 다음 literal이 없으면 남은 텍스트 전체
                    replaced_text = orig_line[orig_pos:]
                    orig_pos = len(orig_line)

                replaced_text = replaced_text.strip()
                if replaced_text:
                    mapping[replaced_text] = part

            else:
                # literal인 경우, 원본에서 위치 업데이트
                found_idx = orig_line.find(part, orig_pos)
                if found_idx != -1:
                    orig_pos = found_idx + len(part)

    return mapping


# 비식별화 함수 정의
def process_interface(index):
    data = dataset["test"][index]
    input_data = tokenizer.apply_chat_template(data["messages"][:2], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_data, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=512, 
        temperature=0.1,
        pad_token_id=tokenizer.eos_token_id)
    anonymized_output = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    mapping =extract_placeholder_mapping(data["messages"][1]["content"], 
                                         anonymized_output, 
                                         allowed_types=(
                "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
                "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID"
            )
        )
    final_mapping = "\n".join([f"{orig_val} -> {ph}" for orig_val, ph in mapping.items()])
    return anonymized_output, final_mapping


# Gradio UI 구성
with gr.Blocks() as demo:
    data_index = gr.Dropdown(label="데이터 선택", choices=[i for i in range(len(dataset["test"]))])
    select_btn = gr.Button("데이터 불러오기")

    with gr.Row():
        original_input = gr.Textbox(label="입력 데이터 (원본)", lines=10)
        model_output = gr.Textbox(label="모델 예측 결과", lines=10)

    with gr.Row():
        to_anonymize = gr.Textbox(label="비식별화할 내용", lines=10)
        anonymized_data = gr.Textbox(label="비식별화된 데이터", lines=10)

    predict_btn = gr.Button("모델 예측하기")

    select_btn.click(
        fn=select_data,
        inputs=[data_index],
        outputs=[original_input, to_anonymize]
    )

    predict_btn.click(
        fn=process_interface,
        inputs=[data_index],
        outputs=[model_output, anonymized_data]
    )

# Gradio 실행
demo.launch(share=True)


/home/admin/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 807/807 [00:00<00:00, 8036.86 examples/s]


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://7c6b65412e5ff05cab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
